In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.optimizers import Adam
import os
pd.options.mode.chained_assignment = None
np.random.seed(2017)

Using TensorFlow backend.
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train_data = pd.read_csv('../../data/spooky-author/download/train.csv')
test_data = pd.read_csv('../../data/spooky-author/download/test.csv')
mapper = {'EAP':0, 'HPL':1, 'MWS':2}
train_data['author'] = train_data['author'].map(lambda x : mapper[x])

In [3]:
train_texts = train_data['text'].values.tolist()
score_texts = test_data['text'].values.tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts + score_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(score_texts)
word_index = tokenizer.word_index
print('words:', len(word_index))
train_seq = pad_sequences(train_sequences, maxlen=90)
score_seq = pad_sequences(test_sequences, maxlen=90)
labels = to_categorical(train_data['author'].values.tolist())
print('data:', train_seq.shape, score_seq.shape)
print('label:', labels.shape)

words: 29451
data: (19579, 90) (8392, 90)
label: (19579, 3)


### train model

In [4]:
indices = np.arange(train_seq.shape[0])
np.random.shuffle(indices)
train_seq = train_seq[indices]
labels = labels[indices]
nb_validation_samples = int(0.33 * train_seq.shape[0])
x_train = train_seq[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = train_seq[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [5]:
embeddings_index = {}

f = open(os.path.join('../../src/pip/glove/glove.6B.50d.txt'))

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

del embeddings_index

embedding_layer = Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], input_length=90, trainable=True)

In [6]:
model = Sequential()
model.add(embedding_layer)
model.add(GlobalAveragePooling1D())
model.add(Dense(3, activation='softmax'))
optimizer = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [7]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 3
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0001
model.fit(x_train, y_train, **params, )

Train on 13118 samples, validate on 6461 samples
Epoch 1/3
13118/13118 [==============================] - 12s 949us/step - loss: 1.0823 - acc: 0.4002 - val_loss: 1.0681 - val_acc: 0.4284
Epoch 2/3
13118/13118 [==============================] - 12s 942us/step - loss: 1.0609 - acc: 0.4271 - val_loss: 1.0508 - val_acc: 0.4479
Epoch 3/3
13118/13118 [==============================] - 12s 943us/step - loss: 1.0429 - acc: 0.4562 - val_loss: 1.0333 - val_acc: 0.4705


In [8]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 10
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.001
model.fit(x_train, y_train, **params)

Train on 13118 samples, validate on 6461 samples
Epoch 1/10
13118/13118 [==============================] - 12s 942us/step - loss: 1.0233 - acc: 0.4835 - val_loss: 1.0146 - val_acc: 0.4990
Epoch 2/10
13118/13118 [==============================] - 12s 941us/step - loss: 1.0018 - acc: 0.5154 - val_loss: 0.9941 - val_acc: 0.5264
Epoch 3/10
13118/13118 [==============================] - 12s 944us/step - loss: 0.9783 - acc: 0.5479 - val_loss: 0.9721 - val_acc: 0.5484
Epoch 4/10
13118/13118 [==============================] - 12s 944us/step - loss: 0.9530 - acc: 0.5739 - val_loss: 0.9492 - val_acc: 0.5793
Epoch 5/10
13118/13118 [==============================] - 12s 944us/step - loss: 0.9266 - acc: 0.6037 - val_loss: 0.9255 - val_acc: 0.6002
Epoch 6/10
13118/13118 [==============================] - 12s 942us/step - loss: 0.8993 - acc: 0.6230 - val_loss: 0.9024 - val_acc: 0.6318
Epoch 7/10
13118/13118 [==============================] - 12s 938us/step - loss: 0.8717 - acc: 0.6531 - val_loss: 0.8

In [9]:
params = {}
params['validation_data'] = (x_val, y_val)
params['batch_size'] = 8
params['epochs'] = 10
params['callbacks'] = [EarlyStopping(patience=2, monitor='val_loss')]
model.optimizer.lr = 0.0005
model.fit(x_train, y_train, **params)

Train on 13118 samples, validate on 6461 samples
Epoch 1/10
13118/13118 [==============================] - 12s 939us/step - loss: 0.7650 - acc: 0.7192 - val_loss: 0.7895 - val_acc: 0.6895
Epoch 2/10
13118/13118 [==============================] - 12s 938us/step - loss: 0.7404 - acc: 0.7330 - val_loss: 0.7688 - val_acc: 0.6982
Epoch 3/10
13118/13118 [==============================] - 12s 938us/step - loss: 0.7165 - acc: 0.7453 - val_loss: 0.7496 - val_acc: 0.7100
Epoch 4/10
13118/13118 [==============================] - 12s 940us/step - loss: 0.6937 - acc: 0.7560 - val_loss: 0.7315 - val_acc: 0.7175
Epoch 5/10
13118/13118 [==============================] - 12s 940us/step - loss: 0.6717 - acc: 0.7664 - val_loss: 0.7140 - val_acc: 0.7271
Epoch 6/10
13118/13118 [==============================] - 12s 939us/step - loss: 0.6507 - acc: 0.7763 - val_loss: 0.6977 - val_acc: 0.7310
Epoch 7/10
13118/13118 [==============================] - 12s 938us/step - loss: 0.6304 - acc: 0.7837 - val_loss: 0.6

### score model

In [10]:
train_predict = pd.DataFrame(model.predict_proba(train_seq), columns=['k1','k2','k3'])
train_class = pd.DataFrame(model.predict_classes(train_seq), columns=['keras'])
test_predict = pd.DataFrame(model.predict_proba(score_seq), columns=['k1','k2','k3'])
test_class = pd.DataFrame(model.predict_classes(score_seq), columns=['keras'])
train_keras = pd.concat([train_data[['id']], train_predict, train_class], axis=1)
test_keras = pd.concat([test_data[['id']], test_predict, test_class], axis=1)
print('train',train_keras.shape)
print('test', test_keras.shape)
train_keras.to_csv('../../data/spooky-author/data/train_nn_score.csv', index=False)
test_keras.to_csv('../../data/spooky-author/data/test_nn_score.csv', index=False)

train (19579, 5)
test (8392, 5)
